In [1]:
from etc.init import *

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
[monochrome_palette]: value was greater than 1 and was clipped.



import copy
import os.path
import itertools
from collections import Iterable, OrderedDict, namedtuple, deque
import numpy as np
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image
from matplotlib.gridspec import GridSpec
import pymc3 as pymc
import pandas as pd
from pandas import DataFrame
from parameters import ParameterSet

from importlib import reload

import theano_shim as shim

import mackelab as ml
import mackelab.smttk as smttk
import mackelab.iotools
import mackelab.pymc3
import mackelab.parameters
import mackelab.plot
ml.parameters.Transform.namespaces['shim'] = shim

import sinn
import sinn.histories as histories
import sinn.models
from sinn.optimize.gradient_descent import FitCollection
import sinn.analyze as anlz
from sinn.analyze.axisdata import LogLikelihood, Likelihood, Probability
from sinn.analyze.axis import Axis

import fsGIF.core as core
import fsGIF.fsgif_model as fsgif_model
from fsGIF.fsgif_model

In [2]:
records = smttk.get_records(recordstore, 'fsGIF')

In [27]:
records.summary.head(15)

,# records,reason,outcome,tags,main_file,avg . duration
20200204-114454,51,fig7_trainI_inferred-meso,,,../code/fsGIF/generate_activity.py,1h 23m 07s
20200204-114338,18,fig_cstI_true-micro,,,../code/fsGIF/generate_spikes.py,3h 52m 34s
20200204-114226,61,fig7_trainI_true-meso,,,../code/fsGIF/generate_activity.py,1h 09m 18s
20200202-142255,100,fig5_stepI_hetero_inferred-full_meso,,,../code/fsGIF/generate_activity.py,0h 41m 10s
20200202-140422,100,fig_cstI_inferred_meso_25-fits,,,../code/fsGIF/generate_activity.py,0h 54m 30s
20200202-140250,1,constant_input__0.5,,,../code/fsGIF/generate_input.py,0h 00m 05s
20200202-135810,1,constant_input__0.5,,,../code/fsGIF/generate_input.py,0h 00m 06s
20200127-111900,43,fig8_rampI_true-micro,,,../code/fsGIF/generate_spikes.py,6h 02m 06s
20200127-111754,100,fig8_rampI_true-meso,,,../code/fsGIF/generate_activity.py,1h 17m 04s
20200127-111650,100,fig8_rampI_inferred-meso,,,../code/fsGIF/generate_activity.py,1h 06m 50s


In [29]:
records.filter.script('spikes').filter(lambda rec: len(rec.parameters.model.N) == 4).summary

,# records,reason,outcome,tags,main_file,avg . duration
20200127-111900,49,fig8_rampI_true-micro,,,../code/fsGIF/generate_spikes.py,6h 17m 11s
20200127-105335,50,fig8_OU-I_true-micro,,,../code/fsGIF/generate_spikes.py,6h 08m 38s
20200127-104743,49,fig7_trainI_true-micro,,,../code/fsGIF/generate_spikes.py,6h 15m 39s
20190708-092937,2,λbar_true-params,,,../code/fsGIF/generate_spikes.py,1h 44m 22s
20190705-091740,1,λbar_true-params,,,../code/fsGIF/generate_spikes.py,1h 43m 17s
20190704-145238,1,λbar_true-params,,,../code/fsGIF/generate_spikes.py,1h 42m 04s
20190627-232628,1,λbar_true-params,,,../code/fsGIF/generate_spikes.py,1h 44m 40s
20190627-164646,1,spikes_brown-noise-4pop,,,../code/fsGIF/generate_spikes.py,1h 45m 16s
20181121-002533,1,λbar_true-params_seed-350,,,fsGIF/fsGIF/generate_spikes.py,1h 38m 07s
20181120-173908,1,λbar_true-params,,,fsGIF/fsGIF/generate_spikes.py,2h 37m 07s
